## Homework 3: ``openpyxl``

### Overview

Excel is a powerful tool, not as powerful as R or Python, but it will *always* have its place in the world of business. That is why I am taking a spreadsheet modelling class this semester (Management Science with Professor Roumani, great class). During my internship rotations with FCA, I found myself always deep in a spreadsheet of some kind or another. Last summer I even integrated some VBA automation into a weekly filing to make my manager's life easier after I left. When I return to FCA in July, I'm sure it will be much of the same story; however, I would like to integrate a more advanced tool with Excel, such as Python or R. I was looking into ``rexcel`` last summer; this is an add-in for Excel that integrates R. It seemed to be more GUI-based, but it looked interesting. I never got a chance to check it out though.

Now I am looking for Excel integration in Python. I reviewed your video on ``xlrd``, ``xlwt`` and ``xlutils``. After watching it, I was quickly refreshed on your example. As you noted, it seems ``openpyxl`` is the rising package for Excel manipulation in Python. Last time we spoke you also mentioned that Python could be integrated with Excel natively, which I'm sure would make a lot of people very happy. Fingers crossed!

This homework submission will be entirely derived from chapter 12 of [*Automate the Boring Stuff*](https://automatetheboringstuff.com/chapter12/) (*AtBS*). The first section will be me working through the book examples, while later I will tackle the questions and suggested problems from the text.

### Examples

First, let's import ``openpyxl`` and learn a bit more about it.

In [16]:
import openpyxl

Let's load a workbook. This would be where ``xlrd`` would be needed previously.

In [17]:
wb = openpyxl.load_workbook('data/example.xlsx')
type(wb)

openpyxl.workbook.workbook.Workbook

We have created a spreadsheet object of type "Workbook". This is an Excel workbook.  

I haven't previewed this file so let's see how it is structured.

In [18]:
# wb.get_sheet_names() is the suggested way to do this by the text, but it has been deprecated
# Below is the new way to do this 
wb.sheetnames

['Sheet1', 'Sheet2', 'Sheet3']

In [19]:
# wb.get_sheet_by_name('Sheet1') is deprecated, below is the revised version
sheet1 = wb['Sheet1']
print('Cols:\t', sheet1.max_column)
print('Rows:\t', sheet1.max_row)

Cols:	 3
Rows:	 7


Now that we know what the general structure of the workbook is, let's look at the individual cells.

In [20]:
sheet1['B1']

<Cell 'Sheet1'.B1>

Okay, it seems we just accessed the cell. But this is not what we want. What is in the cell? It turns out, similar to VBA, were are looking for its *value*.

In [21]:
sheet1['B1'].value

'Apples'

Can we change the value using Python? Well let's see.

In [22]:
sheet1['B1'] = 'Durian'
sheet1['B1'].value

'Durian'

What other attributes do cells have?

In [23]:
print('Col:\t', sheet1['B1'].column)
print('Row:\t', sheet1['B1'].row)
print('Coord:\t', sheet1['B1'].coordinate)

Col:	 B
Row:	 1
Coord:	 B1


Let's see what is in each column.

In [24]:
for row in sheet1.iter_rows():
    print([cell.value for cell in row])

[datetime.datetime(2015, 4, 5, 13, 34, 2), 'Durian', 73]
[datetime.datetime(2015, 4, 5, 3, 41, 23), 'Cherries', 85]
[datetime.datetime(2015, 4, 6, 12, 46, 51), 'Pears', 14]
[datetime.datetime(2015, 4, 8, 8, 59, 43), 'Oranges', 52]
[datetime.datetime(2015, 4, 10, 2, 7), 'Apples', 152]
[datetime.datetime(2015, 4, 10, 18, 10, 37), 'Bananas', 23]
[datetime.datetime(2015, 4, 10, 2, 40, 46), 'Strawberries', 98]


It appears that the first column is a datetime value, the next is a type of fruit, and the third is a numeric column.  

Alternatively, it is often easier to iterate using the integer values of the columns instead of the alphabetic ones.

In [25]:
for i in range(1, sheet1.max_column):
    print(i, sheet1.cell(row = 1, column = i).value)

1 2015-04-05 13:34:02
2 Durian


``openpyxl`` can also write Excel files, like with ``xlwt``. Let's create a blank workbook.

In [26]:
new_book = openpyxl.Workbook()
new_book.sheetnames

['Sheet']

We created an empty workbook that only has one sheet named "Sheet". Let's add some content to the file and then write it out.

In [27]:
new_book['Sheet']['A1'].value = 100
new_book.save('test.xlsx')

You can view the new file [here](test.xlsx), perhaps not in Jupyter Lab though.

``openpyxl`` also supports more advanced Excel editing. One big one is changing the font.

In [28]:
from openpyxl.styles import Font

``Font`` supports three kwargs: ``name`` (str), ``size`` (int), ``bold`` (bool), and ``italic``(bool). Those familiar with Excel can quickly adjust the style of a given cell to that of their choosing. Let's move back to the original workbook and edit a cell's format.

In [29]:
sheet1['B1'].font = Font(name = 'Courier New', size = 16,
                         bold = True, italic = False)
sheet1['B1'] = 'Durian'

One thing that is missing from the example file is column headers in the first row. Let's add this to Sheet1. Similarly, rows can also be deleted. While we're at it, let's create a freeze pane.

In [30]:
sheet1.insert_rows(1)
sheet1['A1'] = 'Date'
sheet1['B1'] = 'Type'
sheet1['C1'] = 'Frequency'
sheet1.freeze_panes = 'B2'

``openpyxl`` can also alter formulas. Much like VBA, formulas are just strings. Let's sum up the values in the third column of Sheet1.

In [31]:
sheet1['B9'] = 'Total'
sheet1['C9'] = '=sum(C2:C8)'

Cells can also be merged using ``openpyxl``. Let's try it out.

In [32]:
sheet1['E1'] = 'Three merged cells'
sheet1.merge_cells('E1:G1')

The dimensions of rows and columns can also be edited:

In [33]:
sheet1.row_dimensions[1].height = 70
sheet1.column_dimensions['B'].width = 20

This module can also create charts; however, it cannot load them. So if a spreadsheet is loaded with a chart and then saved, the chart is effectively deleted.  

Charts require a reference object, which is the range of cells to be plotted. In this case, we have two series objects: the categories and the actual numeric data.

In [34]:
# create reference objects
fruit = openpyxl.chart.Reference(sheet1, min_col = 2, min_row = 2, max_col = 2, max_row = 8)
data = openpyxl.chart.Reference(sheet1, min_col = 3, min_row = 2, max_col = 3, max_row = 8)

# create chart
chart = openpyxl.chart.BarChart()
chart.title = 'Frequencies of Fruits'
chart.x_axis.title = 'Type'
chart.y_axis.title = 'Frequency'
chart.add_data(data)
chart.set_categories(fruit)
sheet1.add_chart(chart, 'C12')

# save sheet
wb.save('example_copy.xlsx')

Creating the reference objects is somewhat confusing; however, once that is complete, editing the chart is very similar to the ``matplotlib`` API.  

This is everything that *AtBS* covers on Excel. You can view the edited example [xlsx file](example_copy.xlsx) to confirm the changes from the original. 

### Project: Reading Data from a Spreadsheet

The texts suggests creating a file that essentially creates a pivot table using ``openpyxl``, except the code is very verbose and unneccesary, especially in light of ``pandas``. Additionally, the output file is a .py file that can only be accessed using Python. 

Here is my interpretation of the problem:

In [36]:
import pandas as pd

In [38]:
wb = openpyxl.load_workbook('data/censuspopdata.xlsx')
ws = wb['Population by Census Tract']

data = ws.values
header = next(data)[0:]

df = pd.DataFrame(data, columns=header)

It is simpler than I thought to turn an Excel file into a ``pandas`` data frame.

In [64]:
df.head(10)

,CensusTract,State,County,POP2010
0,01001020100,AL,Autauga,1912
1,01001020200,AL,Autauga,2170
2,01001020300,AL,Autauga,3373
3,01001020400,AL,Autauga,4386
4,01001020500,AL,Autauga,10766
5,01001020600,AL,Autauga,3668
6,01001020700,AL,Autauga,2891
7,01001020801,AL,Autauga,3081
8,01001020802,AL,Autauga,10435
9,01001020900,AL,Autauga,5675


In [65]:
df2 = df[['State','County','POP2010']].groupby(by = ['State', 'County']).agg(['sum', 'count'])
df2.reset_index(inplace = True)
df2.columns = ['State','County','Census_Tract', 'Population']
df2.head(10)

,State,County,Census_Tract,Population
0,AK,Aleutians East,3141,1
1,AK,Aleutians West,5561,2
2,AK,Anchorage,291826,55
3,AK,Bethel,17013,3
4,AK,Bristol Bay,997,1
5,AK,Denali,1826,1
6,AK,Dillingham,4847,2
7,AK,Fairbanks North Star,97581,19
8,AK,Haines,2508,1
9,AK,Hoonah-Angoon,2150,2


And it is that simple! I also created a [Python script](project.py) that performs the same task, except it is generalized. This way in 2020, the same analysis can be performed much easier.

You can compare my script to the one supplied by the text [here](data/readCensusExcel.py). I think you'll agree mine is simpler and the output is more applicable to the problem at hand.

### Conclusions

Going into this section, I believed ``openpyxl`` was the answer to all programmatic tasks in Excel; however, this is not the case. That is not to say that is not powerful, it is just more useful for repetitive tasks and data cleanup. I believe I will utilize it moving forward; however, VBA will likely still be required at times. Hopefully, the Python-Excel integration kills VBA.

**Next Steps:**  
My final project is the last submission for this class. It will be an extension of the JV analysis done previously. It will feature a much more robust EDA section with greater analysis. I am not especially interested in building a classification model; the ultimate goal is to understand why JV is so great so consistently. To understand the this, he will need to be compared to other starting pitchers.